In [1]:
from z3 import Int, Bool, Ints, Bools, Solver, Or, Not, And, BV2Int, BitVec, BitVecs, BitVecVal, BitVec, Concat, Extract, IntVal, Implies
from typing import List

In [2]:
## Useful stuff from intel_altera
def vector_to_int(vector):
    result = 0
    for i, x in enumerate(vector):
        result += x * (2 ** i)
    return result


def int_to_vector(x, vector_dim):
    result = []
    for _ in range(vector_dim):
        result.append(x & 1)
        x >>= 1
    return result

In [3]:
## Important cell do not remove

def bool2int(x: bool) -> int:
    return x # 1 if x else 0


def bitvec2int(bitvec : List[bool]) -> int:
    result = 0
    for i, xi in enumerate(bitvec):
        xi = bool2int(xi)
        result += xi * (2 ** i)
    return result


def int2bitvec(n : int, i : int) -> List[bool]:
    result = []
    for _ in range(n):
        if (i > 0):
            result.append(i & 1)
            i >>= 1
        else:
            result.append(0)
    return result


"""
f - битовый вектор длины 2 ** n
x - битовый вектор длины n
"""
def evaluate_fun(f: List[bool], x: List[bool]) -> bool:
    n = len(x)
    if (len(f) != 2 ** n):
        raise Exception("Bad arguments")
    x = bitvec2int(x)
    return f[x]

def z3_evaluate_fun(f, x):
    return f[BV2Int(x)]

In [4]:
def right_inclusive_range(start, end):
    return range(start + 1, end + 1)

def create_bitvec(name, bitlength):
    global_vars[name] = BitVec(name, bitlength)
    
def create_int(name):
    global_vars[name] = Int(name)
    
def add_assert(*asserts):
    for a in asserts:
        global_asserts.append(a)

In [5]:
complexity = 1

global_vars = {}
global_asserts = []

# majority function
majority_bitvector = 232


# gate function
gate_bitvector = majority_bitvector
gate_arity = 3
gate_bitlength = 2 ** gate_arity   # 8

global_vars['gate'] = BitVec('gate', gate_bitlength)
global_vars['gate_value'] = BitVecVal(gate_bitvector, gate_bitlength)


# synthesized function
f_bitvector = 232
f_arity = 3
f_bitlength = 2 ** f_arity

# global_vars['f'] = BitVec('f', f_bitlength)
global_vars['f'] = BitVecVal(f_bitvector, f_bitlength)

create_bitvec('output_polarity', 1)

for gate in right_inclusive_range(f_arity, f_arity + complexity):
    for gate_input in right_inclusive_range(0, gate_arity):
        create_bitvec(f'gate_input_polarity_{gate}_{gate_input}', 1)
        create_int(f'gate_input_number_{gate}_{gate_input}')

for f_input_value in range(f_bitlength):
    # bt0 = 0
    create_bitvec(f'gate_output_value_{f_input_value}_0', 1)
    
    # bti - f input values (xi, 1 <= i <= n - f inputs)
    for gate in right_inclusive_range(0, f_arity):
        create_bitvec(f'gate_output_value_{f_input_value}_{gate}', 1)
    
    for gate in right_inclusive_range(f_arity, f_arity + complexity):
        # bti - gate output values (xi, n < i <= n + r - gates)
        create_bitvec(f'gate_output_value_{f_input_value}_{gate}', 1)
        
        # atic - gate input values
        for gate_input in right_inclusive_range(0, gate_arity):
            create_bitvec(f'gate_input_value_{f_input_value}_{gate}_{gate_input}', 1)
    


In [6]:
# sorted(global_vars.items())

In [7]:
for gate in right_inclusive_range(f_arity, f_arity + complexity):
    for gate_input in right_inclusive_range(0, gate_arity):        
        s = global_vars[f'gate_input_number_{gate}_{gate_input}']
        
        # нет циклов
        add_assert(s < gate)

        # номера операндов неотрицательны
        add_assert(s > 0)
   

    # TODO: generalize
    # номера операндов упорядоченны
    s1 = global_vars[f'gate_input_number_{gate}_1']
    s2 = global_vars[f'gate_input_number_{gate}_2']
    s3 = global_vars[f'gate_input_number_{gate}_3']
    
    add_assert(s1 < s2, s2 < s3)
    
    
    # TODO: generalize
    # (Оптимиз.) не более одного операнда инвертировано
    p1 = global_vars[f'gate_input_polarity_{gate}_1']
    p2 = global_vars[f'gate_input_polarity_{gate}_2']
    p3 = global_vars[f'gate_input_polarity_{gate}_3']
    
    add_assert(((p1 | p2) & (p2 | p3) & (p1 | p3)) == 1)
    
    # TODO: убрать
    # add_assert(p1 == 1, p2 == 1, p3 == 1)
    

for f_input_value in range(f_bitlength):
    bt0 = global_vars[f'gate_output_value_{f_input_value}_0']
    add_assert(bt0 == 0)
    
    bt_out = global_vars[f'gate_output_value_{f_input_value}_{f_arity + complexity}']
    p_out = global_vars['output_polarity']
    f = global_vars['f']
    i = f_input_value
    
    add_assert(bt_out == ~p_out + Extract(i, i, f))
    
    # выходы bi элементов xi, 1 <= i <= f_arity === входы функции f
    for gate in right_inclusive_range(0, f_arity):
        xti = global_vars[f'gate_output_value_{f_input_value}_{gate}']
        add_assert(xti == int2bitvec(f_arity, f_input_value)[gate - 1])
    
    for gate in right_inclusive_range(f_arity, f_arity + complexity):
        # TODO: generalize
        a1 = global_vars[f'gate_input_value_{f_input_value}_{gate}_1']
        a2 = global_vars[f'gate_input_value_{f_input_value}_{gate}_2']
        a3 = global_vars[f'gate_input_value_{f_input_value}_{gate}_3']
        
        b = global_vars[f'gate_output_value_{f_input_value}_{gate}']
        add_assert(b == ((a1 | a2) & (a1 | a3) & (a2 | a3)))

        for gate_input in right_inclusive_range(0, gate_arity):
            a = global_vars[f'gate_input_value_{f_input_value}_{gate}_{gate_input}']
            p = global_vars[f'gate_input_polarity_{gate}_{gate_input}']
            s = global_vars[f'gate_input_number_{gate}_{gate_input}']
            
            for j in range(1, gate):
                b = global_vars[f'gate_output_value_{f_input_value}_{j}']
                add_assert(Implies(s == j, a == b + ~p))
            

In [8]:
s = Solver()
s.add([i for i in global_asserts])
s.check()

sat

In [9]:
model = s.model()
d = {i.name() : model.get_interp(i) for i in model}
sorted(d.items())

[('gate_input_number_4_1', 1),
 ('gate_input_number_4_2', 2),
 ('gate_input_number_4_3', 3),
 ('gate_input_polarity_4_1', 1),
 ('gate_input_polarity_4_2', 1),
 ('gate_input_polarity_4_3', 1),
 ('gate_input_value_0_4_1', 0),
 ('gate_input_value_0_4_2', 0),
 ('gate_input_value_0_4_3', 0),
 ('gate_input_value_1_4_1', 1),
 ('gate_input_value_1_4_2', 0),
 ('gate_input_value_1_4_3', 0),
 ('gate_input_value_2_4_1', 0),
 ('gate_input_value_2_4_2', 1),
 ('gate_input_value_2_4_3', 0),
 ('gate_input_value_3_4_1', 1),
 ('gate_input_value_3_4_2', 1),
 ('gate_input_value_3_4_3', 0),
 ('gate_input_value_4_4_1', 0),
 ('gate_input_value_4_4_2', 0),
 ('gate_input_value_4_4_3', 1),
 ('gate_input_value_5_4_1', 1),
 ('gate_input_value_5_4_2', 0),
 ('gate_input_value_5_4_3', 1),
 ('gate_input_value_6_4_1', 0),
 ('gate_input_value_6_4_2', 1),
 ('gate_input_value_6_4_3', 1),
 ('gate_input_value_7_4_1', 1),
 ('gate_input_value_7_4_2', 1),
 ('gate_input_value_7_4_3', 1),
 ('gate_output_value_0_0', 0),
 ('gate_o

In [10]:
import re

gate_output_pattern = re.compile(r'gate_output_value_(?P<f_input_value>[0-9]+?)_(?P<gate>[0-9]+?)')
gate_input_pattern = re.compile(r'gate_input_value_(?P<f_input_value>[0-9]+)_(?P<gate>[0-9]+)_(?P<gate_input>[0-9]+)')
table = { 'raw_input_1' : [], 'raw_input_2' : [], 'raw_input_3' : [], 'p_input_1' : [], 'p_input_2' : [], 'p_input_3' : [], 'output' : []}
for name, value in sorted(d.items()):

    m = gate_input_pattern.match(name)
#    print(m)
    if not m is None:
        f_input_value = int(m.group('f_input_value'))      
        gate = int(m.group('gate'))
        gate_input = int(m.group('gate_input'))
        table[f'p_input_{gate_input}'].append(value)

    
    
    m = gate_output_pattern.match(name)
#    print(m)
    if not m is None:
        f_input_value = f_input_value = int(m.group('f_input_value'))
        gate = int(m.group('gate'))
        if gate == 4:
            table['output'].append(value)
        elif gate >= 1 and gate <= 3:
            table[f'raw_input_{gate}'].append(value)

    

In [11]:
table

{'raw_input_1': [0, 1, 0, 1, 0, 1, 0, 1],
 'raw_input_2': [0, 0, 1, 1, 0, 0, 1, 1],
 'raw_input_3': [0, 0, 0, 0, 1, 1, 1, 1],
 'p_input_1': [0, 1, 0, 1, 0, 1, 0, 1],
 'p_input_2': [0, 0, 1, 1, 0, 0, 1, 1],
 'p_input_3': [0, 0, 0, 0, 1, 1, 1, 1],
 'output': [0, 0, 0, 1, 0, 1, 1, 1]}

In [12]:
s1 = Solver()
x = BitVec('x', 8)
s1.add(x == bitvec2int([0, 0, 0, 1, 0, 1, 1, 1]))
s1.check()

sat

In [13]:
s1.model()

[x = 232]

In [14]:
bitvec2int([0, 0, 0, 1, 0, 1, 1, 1])

232

In [15]:
int2bitvec(8, 23)

[1, 1, 1, 0, 1, 0, 0, 0]